In [1]:
import sys
import os
import netsquid as ns
import pydynaa
import pandas
import numpy as np
import math
import matplotlib.pyplot as plt

from numpy import random
from itertools import combinations

from netsquid.nodes import Node, Connection, Network

from netsquid.protocols import Protocol, NodeProtocol, Signals

from netsquid.util.datacollector import DataCollector

import netsquid.qubits.ketstates as ks

from netsquid.examples.teleportation import example_network_setup
from netsquid.qubits import qubitapi as qapi
from netsquid.qubits.qubitapi import ops
from netsquid.qubits import ketstates, StateSampler, QFormalism, dmtools

from netsquid.components import ClassicalChannel, QuantumChannel, QuantumProgram, QuantumProcessor, INSTR_MEASURE
from netsquid.components.qsource import QSource, SourceStatus
from netsquid.components.models import DelayModel, FixedDelayModel, FibreDelayModel, FibreLossModel, DephaseNoiseModel, DepolarNoiseModel
from netsquid.components.qprocessor import QuantumProcessor, PhysicalInstruction
from netsquid.components.instructions import INSTR_ROT_Z, INSTR_INIT, INSTR_EMIT, IMeasureFaulty

import netsquid.components.instructions as instr

import import_ipynb
from NoiseMods import EmissionNoiseModel, CollectiveDephasingNoiseModel
from Instructions import IonTrapMultiQubitRotation, IonTrapMSGate, INSTR_INIT_BELL
from InitPhoton import SendPhoton
from programs import emit_prog

importing Jupyter notebook from NoiseMods.ipynb
importing Jupyter notebook from Instructions.ipynb
importing Jupyter notebook from InitPhoton.ipynb
importing Jupyter notebook from programs.ipynb


In [2]:
def setup(collection_efficiency, fiber_length, attenuation):
    
    ns.set_qstate_formalism(QFormalism.DM)
    
    ion_trap = SendPhoton(num_positions=1, collection_efficiency=collection_efficiency)
    
    ion_trap.add_instruction(instruction=INSTR_INIT_BELL, duration=0)
    q1, q2 = qapi.create_qubits(2)
    ns.qubits.operate(q1, ops.H)
    ns.qubits.operate([q1, q2], ops.CNOT)
    
    loss_model = FibreLossModel(p_loss_length=attenuation, p_loss_init=.1e-2)
    fiber = QuantumChannel(name="fiber", length=fiber_length,
                           models={"quantum_loss_model": loss_model},
                           transmit_empty_items=True)
    
    fiber_loss_model = FibreLossModel(p_loss_length=attenuation, p_loss_init=.1e-2)
    #setting p_loss_init = .01 (99% usually Ba excitation works)
    #double counting for phenomenon in iontrap thru colleciton_efficiency and here thru p_loss_init
    fiber_delay_model = FibreDelayModel(c=3e8/1.5)
    
    fiber = QuantumChannel(name="fiber", length=fiber_length,
                          models={"quantum_loss_model": fiber_loss_model, 
                                 "delay_model": fiber_delay_model},
                          transmit_empty_items=True)
    
    fiber.ports["send"].connect(ion_trap.ports["qout"])
    collection_port = fiber.ports["recv"]

    # collection_port = ion_trap.ports["qout"]

    return ion_trap, collection_port

In [3]:
def range_with_floats(start, stop, step):
    while stop > start:
        yield start
        start += step

In [4]:
# fiber length = 4 m
x = np.arange(0.001, 0.010, 0.001)
xstring = "fiber length"

collection_efficiency = .999
fiber_length = 4e-3
attenuation = 30

success_probs = []
success_prob_errors = []
expected_success_probs = []

In [5]:
for fiber_length in x:

    ion_trap, collection_port = setup(collection_efficiency=collection_efficiency,
                                      fiber_length=fiber_length,
                                      attenuation=attenuation)
    #fail_count = 0
    num_tries = 500
    outcomes = []
    for _ in range(num_tries):
        ion_trap.execute_program(emit_prog)
        ns.sim_run()
        emitted_message = collection_port.rx_output()
        emitted_qubit = emitted_message.items[0]
        
        qubit_ion = ion_trap.peek([0])[0]
        qubit_photon = emitted_message.items[0]
        print(qubit_ion, qubit_photon)
        
        if emitted_qubit is None:
            outcomes.append(0)
        else:
            if emitted_qubit.qstate is None:
                outcomes.append
            else:
                #print(ns.qubits.measure(qubit_photon))
                outcomes.append(1)
                
                
        
        #measure_photon_state()
        #^decreases fidelity to .707 fro .9048 if leave in 
       
        
        #adding 60 \mu s delay to account for control software
        # dephase rate = 1 / coherence time = 1 / 300e-6 (for Ba)
        # last parameter is in ns. 60 \mu s = 60e3 ns 
        c_time = 300e-6 
        dephase_rate = 1 / c_time
        ns.qubits.qubitapi.delay_dephase(qubit_ion, dephase_rate, 60e3) #in ns
        
        qubit_yb = ns.qubits.create_qubits(1)
        
        INSTR_MS_GATE(qubit_yb, qubit_ion) 
        
    #fidelity = qapi.fidelity([qubit_ion, qubit_photon], ketstates.b00)  
        #measuring fidelity in this program doesn't make a whole lot of sense but makes for a useful test to evaluate if working as expected
    success_prob = np.mean(outcomes)
    #print(success_prob)
    success_probs.append(success_prob)
    success_prob_error = np.std(outcomes) / math.sqrt(len(outcomes))
    #print(success_prob_errors)
    success_prob_errors.append(success_prob_error)
    expected_success_prob = collection_efficiency * np.power(10, - attenuation * fiber_length / 10)
    #print(expected_success_prob)
    expected_success_probs.append(expected_success_prob)

Qubit('QS#1-0') Qubit('QS#2-0')
Qubit('QS#3-0') Qubit('QS#4-0')
Qubit('QS#5-0') Qubit('QS#6-0')
Qubit('QS#7-0') Qubit('QS#8-0')
Qubit('QS#9-0') Qubit('QS#10-0')
Qubit('QS#11-0') Qubit('QS#12-0')
Qubit('QS#13-0') Qubit('QS#14-0')
Qubit('QS#15-0') Qubit('QS#16-0')
Qubit('QS#17-0') Qubit('QS#18-0')
Qubit('QS#19-0') Qubit('QS#20-0')
Qubit('QS#21-0') Qubit('QS#22-0')
Qubit('QS#23-0') Qubit('QS#24-0')
Qubit('QS#25-0') Qubit('QS#26-0')
Qubit('QS#27-0') Qubit('QS#28-0')
Qubit('QS#29-0') Qubit('QS#30-0')
Qubit('QS#31-0') Qubit('QS#32-0')
Qubit('QS#33-0') Qubit('QS#34-0')
Qubit('QS#35-0') Qubit('QS#36-0')
Qubit('QS#37-0') Qubit('QS#38-0')
Qubit('QS#39-0') Qubit('QS#40-0')
Qubit('QS#41-0') Qubit('QS#42-0')
Qubit('QS#43-0') None
Qubit('QS#45-0') Qubit('QS#46-0')
Qubit('QS#47-0') Qubit('QS#48-0')
Qubit('QS#49-0') Qubit('QS#50-0')
Qubit('QS#51-0') Qubit('QS#52-0')
Qubit('QS#53-0') Qubit('QS#54-0')
Qubit('QS#55-0') Qubit('QS#56-0')
Qubit('QS#57-0') Qubit('QS#58-0')
Qubit('QS#59-0') Qubit('QS#60-0')
Q

Qubit('QS#479-0') Qubit('QS#480-0')
Qubit('QS#481-0') Qubit('QS#482-0')
Qubit('QS#483-0') Qubit('QS#484-0')
Qubit('QS#485-0') Qubit('QS#486-0')
Qubit('QS#487-0') Qubit('QS#488-0')
Qubit('QS#489-0') Qubit('QS#490-0')
Qubit('QS#491-0') Qubit('QS#492-0')
Qubit('QS#493-0') Qubit('QS#494-0')
Qubit('QS#495-0') Qubit('QS#496-0')
Qubit('QS#497-0') Qubit('QS#498-0')
Qubit('QS#499-0') Qubit('QS#500-0')
Qubit('QS#501-0') Qubit('QS#502-0')
Qubit('QS#503-0') Qubit('QS#504-0')
Qubit('QS#505-0') Qubit('QS#506-0')
Qubit('QS#507-0') Qubit('QS#508-0')
Qubit('QS#509-0') Qubit('QS#510-0')
Qubit('QS#511-0') Qubit('QS#512-0')
Qubit('QS#513-0') Qubit('QS#514-0')
Qubit('QS#515-0') Qubit('QS#516-0')
Qubit('QS#517-0') Qubit('QS#518-0')
Qubit('QS#519-0') Qubit('QS#520-0')
Qubit('QS#521-0') Qubit('QS#522-0')
Qubit('QS#523-0') Qubit('QS#524-0')
Qubit('QS#525-0') Qubit('QS#526-0')
Qubit('QS#527-0') Qubit('QS#528-0')
Qubit('QS#529-0') Qubit('QS#530-0')
Qubit('QS#531-0') Qubit('QS#532-0')
Qubit('QS#533-0') Qubit('QS#

Qubit('QS#957-0') Qubit('QS#958-0')
Qubit('QS#959-0') Qubit('QS#960-0')
Qubit('QS#961-0') Qubit('QS#962-0')
Qubit('QS#963-0') Qubit('QS#964-0')
Qubit('QS#965-0') Qubit('QS#966-0')
Qubit('QS#967-0') Qubit('QS#968-0')
Qubit('QS#969-0') Qubit('QS#970-0')
Qubit('QS#971-0') Qubit('QS#972-0')
Qubit('QS#973-0') Qubit('QS#974-0')
Qubit('QS#975-0') Qubit('QS#976-0')
Qubit('QS#977-0') Qubit('QS#978-0')
Qubit('QS#979-0') Qubit('QS#980-0')
Qubit('QS#981-0') Qubit('QS#982-0')
Qubit('QS#983-0') Qubit('QS#984-0')
Qubit('QS#985-0') Qubit('QS#986-0')
Qubit('QS#987-0') Qubit('QS#988-0')
Qubit('QS#989-0') Qubit('QS#990-0')
Qubit('QS#991-0') Qubit('QS#992-0')
Qubit('QS#993-0') Qubit('QS#994-0')
Qubit('QS#995-0') Qubit('QS#996-0')
Qubit('QS#997-0') Qubit('QS#998-0')
Qubit('QS#999-0') Qubit('QS#1000-0')
Qubit('QS#1002-0') Qubit('QS#1003-0')
Qubit('QS#1004-0') Qubit('QS#1005-0')
Qubit('QS#1006-0') Qubit('QS#1007-0')
Qubit('QS#1008-0') Qubit('QS#1009-0')
Qubit('QS#1010-0') Qubit('QS#1011-0')
Qubit('QS#1012-0'

Qubit('QS#1394-0') Qubit('QS#1395-0')
Qubit('QS#1396-0') Qubit('QS#1397-0')
Qubit('QS#1398-0') Qubit('QS#1399-0')
Qubit('QS#1400-0') Qubit('QS#1401-0')
Qubit('QS#1402-0') Qubit('QS#1403-0')
Qubit('QS#1404-0') Qubit('QS#1405-0')
Qubit('QS#1406-0') Qubit('QS#1407-0')
Qubit('QS#1408-0') Qubit('QS#1409-0')
Qubit('QS#1410-0') Qubit('QS#1411-0')
Qubit('QS#1412-0') Qubit('QS#1413-0')
Qubit('QS#1414-0') Qubit('QS#1415-0')
Qubit('QS#1416-0') Qubit('QS#1417-0')
Qubit('QS#1418-0') Qubit('QS#1419-0')
Qubit('QS#1420-0') Qubit('QS#1421-0')
Qubit('QS#1422-0') Qubit('QS#1423-0')
Qubit('QS#1424-0') Qubit('QS#1425-0')
Qubit('QS#1426-0') Qubit('QS#1427-0')
Qubit('QS#1428-0') Qubit('QS#1429-0')
Qubit('QS#1430-0') Qubit('QS#1431-0')
Qubit('QS#1432-0') Qubit('QS#1433-0')
Qubit('QS#1434-0') Qubit('QS#1435-0')
Qubit('QS#1436-0') Qubit('QS#1437-0')
Qubit('QS#1438-0') Qubit('QS#1439-0')
Qubit('QS#1440-0') Qubit('QS#1441-0')
Qubit('QS#1442-0') Qubit('QS#1443-0')
Qubit('QS#1444-0') Qubit('QS#1445-0')
Qubit('QS#14

Qubit('QS#1854-0') Qubit('QS#1855-0')
Qubit('QS#1856-0') Qubit('QS#1857-0')
Qubit('QS#1858-0') Qubit('QS#1859-0')
Qubit('QS#1860-0') Qubit('QS#1861-0')
Qubit('QS#1862-0') Qubit('QS#1863-0')
Qubit('QS#1864-0') Qubit('QS#1865-0')
Qubit('QS#1866-0') Qubit('QS#1867-0')
Qubit('QS#1868-0') Qubit('QS#1869-0')
Qubit('QS#1870-0') Qubit('QS#1871-0')
Qubit('QS#1872-0') Qubit('QS#1873-0')
Qubit('QS#1874-0') Qubit('QS#1875-0')
Qubit('QS#1876-0') Qubit('QS#1877-0')
Qubit('QS#1878-0') Qubit('QS#1879-0')
Qubit('QS#1880-0') Qubit('QS#1881-0')
Qubit('QS#1882-0') Qubit('QS#1883-0')
Qubit('QS#1884-0') Qubit('QS#1885-0')
Qubit('QS#1886-0') Qubit('QS#1887-0')
Qubit('QS#1888-0') Qubit('QS#1889-0')
Qubit('QS#1890-0') Qubit('QS#1891-0')
Qubit('QS#1892-0') Qubit('QS#1893-0')
Qubit('QS#1894-0') Qubit('QS#1895-0')
Qubit('QS#1896-0') Qubit('QS#1897-0')
Qubit('QS#1898-0') Qubit('QS#1899-0')
Qubit('QS#1900-0') Qubit('QS#1901-0')
Qubit('QS#1902-0') Qubit('QS#1903-0')
Qubit('QS#1904-0') Qubit('QS#1905-0')
Qubit('QS#19

...

Qubit('QS#6891-0') Qubit('QS#6892-0')
Qubit('QS#6893-0') Qubit('QS#6894-0')
Qubit('QS#6895-0') Qubit('QS#6896-0')
Qubit('QS#6897-0') Qubit('QS#6898-0')
Qubit('QS#6899-0') Qubit('QS#6900-0')
Qubit('QS#6901-0') Qubit('QS#6902-0')
Qubit('QS#6903-0') Qubit('QS#6904-0')
Qubit('QS#6905-0') Qubit('QS#6906-0')
Qubit('QS#6907-0') Qubit('QS#6908-0')
Qubit('QS#6909-0') Qubit('QS#6910-0')
Qubit('QS#6911-0') Qubit('QS#6912-0')
Qubit('QS#6913-0') Qubit('QS#6914-0')
Qubit('QS#6915-0') Qubit('QS#6916-0')
Qubit('QS#6917-0') Qubit('QS#6918-0')
Qubit('QS#6919-0') Qubit('QS#6920-0')
Qubit('QS#6921-0') Qubit('QS#6922-0')
Qubit('QS#6923-0') Qubit('QS#6924-0')
Qubit('QS#6925-0') Qubit('QS#6926-0')
Qubit('QS#6927-0') None
Qubit('QS#6929-0') Qubit('QS#6930-0')
Qubit('QS#6931-0') Qubit('QS#6932-0')
Qubit('QS#6933-0') Qubit('QS#6934-0')
Qubit('QS#6935-0') Qubit('QS#6936-0')
Qubit('QS#6937-0') None
Qubit('QS#6939-0') Qubit('QS#6940-0')
Qubit('QS#6941-0') Qubit('QS#6942-0')
Qubit('QS#6943-0') Qubit('QS#6944-0')
Qu

Qubit('QS#7340-0') Qubit('QS#7341-0')
Qubit('QS#7342-0') Qubit('QS#7343-0')
Qubit('QS#7344-0') Qubit('QS#7345-0')
Qubit('QS#7346-0') Qubit('QS#7347-0')
Qubit('QS#7348-0') Qubit('QS#7349-0')
Qubit('QS#7350-0') Qubit('QS#7351-0')
Qubit('QS#7352-0') Qubit('QS#7353-0')
Qubit('QS#7354-0') Qubit('QS#7355-0')
Qubit('QS#7356-0') Qubit('QS#7357-0')
Qubit('QS#7358-0') Qubit('QS#7359-0')
Qubit('QS#7360-0') Qubit('QS#7361-0')
Qubit('QS#7362-0') Qubit('QS#7363-0')
Qubit('QS#7364-0') Qubit('QS#7365-0')
Qubit('QS#7366-0') Qubit('QS#7367-0')
Qubit('QS#7368-0') Qubit('QS#7369-0')
Qubit('QS#7370-0') Qubit('QS#7371-0')
Qubit('QS#7372-0') Qubit('QS#7373-0')
Qubit('QS#7374-0') Qubit('QS#7375-0')
Qubit('QS#7376-0') Qubit('QS#7377-0')
Qubit('QS#7378-0') Qubit('QS#7379-0')
Qubit('QS#7380-0') Qubit('QS#7381-0')
Qubit('QS#7382-0') Qubit('QS#7383-0')
Qubit('QS#7384-0') Qubit('QS#7385-0')
Qubit('QS#7386-0') Qubit('QS#7387-0')
Qubit('QS#7388-0') Qubit('QS#7389-0')
Qubit('QS#7390-0') Qubit('QS#7391-0')
Qubit('QS#73

Qubit('QS#7782-0') Qubit('QS#7783-0')
Qubit('QS#7784-0') Qubit('QS#7785-0')
Qubit('QS#7786-0') Qubit('QS#7787-0')
Qubit('QS#7788-0') Qubit('QS#7789-0')
Qubit('QS#7790-0') Qubit('QS#7791-0')
Qubit('QS#7792-0') Qubit('QS#7793-0')
Qubit('QS#7794-0') Qubit('QS#7795-0')
Qubit('QS#7796-0') Qubit('QS#7797-0')
Qubit('QS#7798-0') Qubit('QS#7799-0')
Qubit('QS#7800-0') Qubit('QS#7801-0')
Qubit('QS#7802-0') Qubit('QS#7803-0')
Qubit('QS#7804-0') Qubit('QS#7805-0')
Qubit('QS#7806-0') Qubit('QS#7807-0')
Qubit('QS#7808-0') None
Qubit('QS#7810-0') Qubit('QS#7811-0')
Qubit('QS#7812-0') Qubit('QS#7813-0')
Qubit('QS#7814-0') None
Qubit('QS#7816-0') Qubit('QS#7817-0')
Qubit('QS#7818-0') Qubit('QS#7819-0')
Qubit('QS#7820-0') Qubit('QS#7821-0')
Qubit('QS#7822-0') Qubit('QS#7823-0')
Qubit('QS#7824-0') Qubit('QS#7825-0')
Qubit('QS#7826-0') Qubit('QS#7827-0')
Qubit('QS#7828-0') Qubit('QS#7829-0')
Qubit('QS#7830-0') Qubit('QS#7831-0')
Qubit('QS#7832-0') None
Qubit('QS#7834-0') Qubit('QS#7835-0')
Qubit('QS#7836-0

Qubit('QS#8223-0') None
Qubit('QS#8225-0') Qubit('QS#8226-0')
Qubit('QS#8227-0') Qubit('QS#8228-0')
Qubit('QS#8229-0') Qubit('QS#8230-0')
Qubit('QS#8231-0') Qubit('QS#8232-0')
Qubit('QS#8233-0') Qubit('QS#8234-0')
Qubit('QS#8235-0') Qubit('QS#8236-0')
Qubit('QS#8237-0') Qubit('QS#8238-0')
Qubit('QS#8239-0') Qubit('QS#8240-0')
Qubit('QS#8241-0') Qubit('QS#8242-0')
Qubit('QS#8243-0') Qubit('QS#8244-0')
Qubit('QS#8245-0') Qubit('QS#8246-0')
Qubit('QS#8247-0') Qubit('QS#8248-0')
Qubit('QS#8249-0') Qubit('QS#8250-0')
Qubit('QS#8251-0') Qubit('QS#8252-0')
Qubit('QS#8253-0') Qubit('QS#8254-0')
Qubit('QS#8255-0') Qubit('QS#8256-0')
Qubit('QS#8257-0') Qubit('QS#8258-0')
Qubit('QS#8259-0') Qubit('QS#8260-0')
Qubit('QS#8261-0') Qubit('QS#8262-0')
Qubit('QS#8263-0') Qubit('QS#8264-0')
Qubit('QS#8265-0') Qubit('QS#8266-0')
Qubit('QS#8267-0') Qubit('QS#8268-0')
Qubit('QS#8269-0') None
Qubit('QS#8271-0') None
Qubit('QS#8273-0') Qubit('QS#8274-0')
Qubit('QS#8275-0') Qubit('QS#8276-0')
Qubit('QS#8277-0

Qubit('QS#8675-0') Qubit('QS#8676-0')
Qubit('QS#8677-0') Qubit('QS#8678-0')
Qubit('QS#8679-0') Qubit('QS#8680-0')
Qubit('QS#8681-0') Qubit('QS#8682-0')
Qubit('QS#8683-0') Qubit('QS#8684-0')
Qubit('QS#8685-0') Qubit('QS#8686-0')
Qubit('QS#8687-0') Qubit('QS#8688-0')
Qubit('QS#8689-0') Qubit('QS#8690-0')
Qubit('QS#8691-0') Qubit('QS#8692-0')
Qubit('QS#8693-0') Qubit('QS#8694-0')
Qubit('QS#8695-0') Qubit('QS#8696-0')
Qubit('QS#8697-0') Qubit('QS#8698-0')
Qubit('QS#8699-0') Qubit('QS#8700-0')
Qubit('QS#8701-0') Qubit('QS#8702-0')
Qubit('QS#8703-0') Qubit('QS#8704-0')
Qubit('QS#8705-0') Qubit('QS#8706-0')
Qubit('QS#8707-0') Qubit('QS#8708-0')
Qubit('QS#8709-0') Qubit('QS#8710-0')
Qubit('QS#8711-0') Qubit('QS#8712-0')
Qubit('QS#8713-0') Qubit('QS#8714-0')
Qubit('QS#8715-0') Qubit('QS#8716-0')
Qubit('QS#8717-0') Qubit('QS#8718-0')
Qubit('QS#8719-0') Qubit('QS#8720-0')
Qubit('QS#8721-0') Qubit('QS#8722-0')
Qubit('QS#8723-0') Qubit('QS#8724-0')
Qubit('QS#8725-0') Qubit('QS#8726-0')
Qubit('QS#87

In [6]:
print("fidelity:", fidelity)
plt.figure()
plt.errorbar(x, y=success_probs, yerr=success_prob_errors, label="measured")
plt.plot(x, expected_success_probs, label="expected")
plt.xlabel(xstring)
plt.ylabel("success probability)")
plt.legend()
plt.show()

NameError: name 'fidelity' is not defined

In [ ]:
#ns.qubits.reduced_dm(qubit_ion)

In [ ]:
#ns.qubits.measure(qubit_ion)

In [ ]:
#ns.qubit.reduced_dm(qubit_photon)

In [ ]:
#ns.qubits.measure(qubit_photon)